# Analyze Product Sentiment

In [8]:
import turicreate

# Read product review data

In [9]:
products = turicreate.SFrame('amazon_baby.sframe')

# Explore data

In [10]:
products

Columns:
	name	str
	review	str
	rating	float

Rows: 183531

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
+-------------------------------+-------------------------------+--------+
[183531 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [11]:
products.groupby('name',operations={'count':turicreate.aggregate.COUNT()}).sort('count',ascending=False)

Columns:
	name	str
	count	int

Rows: 32419

Data:
+--------------------------------+-------+
|              name              | count |
+--------------------------------+-------+
| Vulli Sophie the Giraffe T...  |  785  |
| Simple Wishes Hands-Free B...  |  562  |
| Infant Optics DXR-5 2.4 GH...  |  561  |
| Baby Einstein Take Along Tunes |  547  |
| Cloud b Twilight Constella...  |  520  |
| Fisher-Price Booster Seat,...  |  489  |
| Fisher-Price Rainforest Ju...  |  450  |
| Graco Nautilus 3-in-1 Car ...  |  419  |
| Leachco Snoogle Total Body...  |  388  |
| Regalo Easy Step Walk Thru...  |  374  |
+--------------------------------+-------+
[32419 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

# Examine the reivews for the most-reviewed product

In [12]:
giraffe_reviews = products[products['name']=='Vulli Sophie the Giraffe Teether']

In [13]:
giraffe_reviews

Columns:
	name	str
	review	str
	rating	float

Rows: Unknown

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Vulli Sophie the Giraffe T... | He likes chewing on all th... |  5.0   |
| Vulli Sophie the Giraffe T... | My son loves this toy and ... |  5.0   |
| Vulli Sophie the Giraffe T... | There really should be a l... |  1.0   |
| Vulli Sophie the Giraffe T... | All the moms in my moms' g... |  5.0   |
| Vulli Sophie the Giraffe T... | I was a little skeptical o... |  5.0   |
| Vulli Sophie the Giraffe T... | I have been reading about ... |  5.0   |
| Vulli Sophie the Giraffe T... | My neice loves her sophie ... |  5.0   |
| Vulli Sophie the Giraffe T... | What a friendly face!  And... |  5.0   |
| Vulli Sophie the Giraffe T... | We got this just for my so... |  5.0   |
| Vulli Sophie the Giraffe T... | My baby seems to like this... |  3.0   |
+-------------------------------+-------------------------------+--------+
[? rows x 3 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.

In [14]:
len(giraffe_reviews)

785

In [15]:
giraffe_reviews['rating'].show()

Materializing SArray

# Building a sentiment classifier

## Build word count vectors

In [16]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

In [17]:
products

Columns:
	name	str
	review	str
	rating	float
	word_count	dict

Rows: 183531

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+
|           word_count          |
+-------------------------------+
| {'handles': 1.0, 'strippin... |
| {'recommend': 1.0, 'disapp... |
| {'quilt': 1.0, 'the': 1.0,... |
| {'tool': 1.0, 'clever': 1.... |
| {'rock': 1.0, 'many': 1.0,... |
| {'thumb': 1.0, 'or': 1.0, ... |
| {'for': 1.0, 'barnes': 1.0... |
| {'right': 1.0, 'because': ... |
| {'like': 1.0, 'and': 1.0, ... |
| {'in': 1.0, 'pages': 1.0, ... |
+-------------------------------+
[183531 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

# Define what is positive and negative sentiment

In [18]:
products['rating'].show()

Materializing SArray

In [19]:
#ignore all 3*  reviews
products = products[products['rating']!= 3]

In [20]:
#positive sentiment = 4-star or 5-star reviews
products['sentiment'] = products['rating'] >= 4

In [21]:
products

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int

Rows: 166752

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
| Baby Tracker&reg; - Daily ... | I love this journal and ou... |  4.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+
|           word_count          | sentiment |
+-------------------------------+-----------+
| {'recommend': 1.0, 'disapp... |     1     |
| {'quilt': 1.0, 'the': 1.0,... |     1     |
| {'tool': 1.0, 'clever': 1.... |     1     |
| {'rock': 1.0, 'many': 1.0,... |     1     |
| {'thumb': 1.0, 'or': 1.0, ... |     1     |
| {'for': 1.0, 'barnes': 1.0... |     1     |
| {'right': 1.0, 'because': ... |     1     |
| {'like': 1.0, 'and': 1.0, ... |     1     |
| {'in': 1.0, 'pages': 1.0, ... |     1     |
| {'tracker': 1.0, 'now': 1.... |     1     |
+-------------------------------+-----------+
[166752 rows x 5 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [22]:
products['sentiment'].show()

Materializing SArray

In [48]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']
for x in selected_words:
    def awesome_count(value):
        if x in value:
            return value[x] 
        else:
            return 0
    products[x] = products['word_count'].apply(awesome_count)
    print(f"world: {x} value:{products[x].sum()} ")
products

world: awesome value:3892.0 
world: great value:55791.0 
world: fantastic value:1664.0 
world: amazing value:2628.0 
world: love value:41994.0 
world: horrible value:1110 
world: bad value:4183 
world: terrible value:1146.0 
world: awful value:687 
world: wow value:425 
world: hate value:1107 


Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	awesome	float
	great	float
	fantastic	float
	amazing	float
	love	float
	horrible	int
	bad	int
	terrible	float
	awful	int
	wow	int
	hate	int

Rows: 166752

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
| Baby Tracker&reg; - Daily ... | I love this journal and ou... |  4.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+---------+-------+-----------+---------+
|           word_count          | sentiment | awesome | great | fantastic | amazing |
+-------------------------------+-----------+---------+-------+-----------+---------+
| {'recommend': 1.0, 'disapp... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'quilt': 1.0, 'the': 1.0,... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'tool': 1.0, 'clever': 1.... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'rock': 1.0, 'many': 1.0,... |     1     |   0.0   |  1.0  |    0.0    |   0.0   |
| {'thumb': 1.0, 'or': 1.0, ... |     1     |   0.0   |  1.0  |    0.0    |   0.0   |
| {'for': 1.0, 'barnes': 1.0... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'right': 1.0, 'because': ... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'like': 1.0, 'and': 1.0, ... |     1     |   0.0   |  0.0  |    1.0    |   0.0   |
| {'in': 1.0, 'pages': 1.0, ... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'tracker': 1.0, 'now': 1.... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
+-------------------------------+-----------+---------+-------+-----------+---------+
+------+----------+-----+----------+-------+-----+------+
| love | horrible | bad | terrible | awful | wow | hate |
+------+----------+-----+----------+-------+-----+------+
| 1.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |
| 2.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |
| 1.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |
| 2.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |
+------+----------+-----+----------+-------+-----+------+
[166752 rows x 16 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

# Train our sentiment classifier

In [49]:
train_data,test_data = products.random_split(.8,seed=0)

In [55]:
features = selected_words
selected_words_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=features, validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.787926     | 0.847708          | 0.846385            |

| 5         | 6        | 0.912556     | 0.847708          | 0.846385            |

| 6         | 7        | 1.025428     | 0.847708          | 0.846385            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [74]:
selected_words_model.coefficients.sort("value").print_rows(12) 

+-------------+-------+-------+-----------------------+----------------------+
|     name    | index | class |         value         |        stderr        |
+-------------+-------+-------+-----------------------+----------------------+
|   horrible  |  None |   1   |   -2.251335236759093  | 0.08020249388788442  |
|   terrible  |  None |   1   |   -2.223661436085127  | 0.07731736203785755  |
|    awful    |  None |   1   |  -2.0529082040313513  | 0.10099735435259259  |
|     hate    |  None |   1   |  -1.3484407222463124  | 0.07715698604297333  |
|     bad     |  None |   1   |  -0.9914778800650565  | 0.03848428664699063  |
|     wow     |  None |   1   | -0.009538236067678897 |  0.1604641122471166  |
|    great    |  None |   1   |   0.8630655001196618  | 0.018955052444377323 |
|  fantastic  |  None |   1   |   0.8858047568814295  | 0.11167591293399656  |
|   amazing   |  None |   1   |   1.1000933113660283  | 0.09954776260465983  |
|   awesome   |  None |   1   |   1.133534666034145 

In [60]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8463848186404036,
 'auc': 0.6935096220934976,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  159  |
 |      0       |        0        |  371  |
 |      0       |        1        |  4957 |
 |      1       |        1        | 27817 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9157860082304526,
 'log_loss': 0.3962265467087378,
 'precision': 0.8487520595594068,
 'recall': 0.9943165570488991,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 5328 

# Apply the sentiment classifier to better understand the Giraffe reviews

In [61]:
products['predicted_sentiment'] = selected_words_model.predict(products, output_type = 'probability')

In [62]:
products

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	awesome	float
	great	float
	fantastic	float
	amazing	float
	love	float
	horrible	int
	bad	int
	terrible	float
	awful	int
	wow	int
	hate	int
	predicted_sentiment	float

Rows: 166752

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
| Baby Tracker&reg; - Daily ... | I love this journal and ou... |  4.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+---------+-------+-----------+---------+
|           word_count          | sentiment | awesome | great | fantastic | amazing |
+-------------------------------+-----------+---------+-------+-----------+---------+
| {'recommend': 1.0, 'disapp... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'quilt': 1.0, 'the': 1.0,... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'tool': 1.0, 'clever': 1.... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'rock': 1.0, 'many': 1.0,... |     1     |   0.0   |  1.0  |    0.0    |   0.0   |
| {'thumb': 1.0, 'or': 1.0, ... |     1     |   0.0   |  1.0  |    0.0    |   0.0   |
| {'for': 1.0, 'barnes': 1.0... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'right': 1.0, 'because': ... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'like': 1.0, 'and': 1.0, ... |     1     |   0.0   |  0.0  |    1.0    |   0.0   |
| {'in': 1.0, 'pages': 1.0, ... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'tracker': 1.0, 'now': 1.... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
+-------------------------------+-----------+---------+-------+-----------+---------+
+------+----------+-----+----------+-------+-----+------+---------------------+
| love | horrible | bad | terrible | awful | wow | hate | predicted_sentiment |
+------+----------+-----+----------+-------+-----+------+---------------------+
| 1.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.936781924479937  |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.7919288370624453 |
| 2.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.9829620803909053 |
| 1.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.9723186788855891 |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.9002186948093641 |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.7919288370624453 |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.7919288370624453 |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.9022427410616031 |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.7919288370624453 |
| 2.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.9829620803909053 |
+------+----------+-----+----------+-------+-----+------+---------------------+
[166752 rows x 17 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [67]:
giraffe_reviews = products[products['name']== 'Baby Trend Diaper Champ']

In [68]:
giraffe_reviews

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	awesome	float
	great	float
	fantastic	float
	amazing	float
	love	float
	horrible	int
	bad	int
	terrible	float
	awful	int
	wow	int
	hate	int
	predicted_sentiment	float

Rows: Unknown

Data:
+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | Ok - newsflash.  Diapers a... |  4.0   |
| Baby Trend Diaper Champ | My husband and I selected ... |  1.0   |
| Baby Trend Diaper Champ | Excellent diaper disposal ... |  5.0   |
| Baby Trend Diaper Champ | We love our diaper champ. ... |  5.0   |
| Baby Trend Diaper Champ | Two girlfriends and two fa... |  5.0   |
| Baby Trend Diaper Champ | I waited to review this un... |  4.0   |
| Baby Trend Diaper Champ | I have had a diaper genie ... |  1.0   |
| Baby Trend Diaper Champ | I originally put this item... |  5.0   |
| Baby Trend Diaper Champ | I am so glad I got the Dia... |  5.0   |
| Baby Trend Diaper Champ | We had 2 diaper Genie's bo... |  4.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+-----------+---------+-------+-----------+---------+
|           word_count          | sentiment | awesome | great | fantastic | amazing |
+-------------------------------+-----------+---------+-------+-----------+---------+
| {'convenient': 1.0, 'more'... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'system': 1.0, 'try': 1.0... |     0     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'nose': 1.0, 'for': 2.0, ... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'out': 1.0, 'pull': 1.0, ... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'winter': 1.0, 'outside':... |     1     |   0.0   |  0.0  |    0.0    |   1.0   |
| {'mom': 1.0, 'huge': 1.0, ... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'yuck': 1.0, 'clean': 1.0... |     0     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'price': 1.0, 'suggestion... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'best': 1.0, 'that': 1.0,... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'no': 1.0, 'regular': 1.0... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
+-------------------------------+-----------+---------+-------+-----------+---------+
+------+----------+-----+----------+-------+-----+------+---------------------+
| love | horrible | bad | terrible | awful | wow | hate | predicted_sentiment |
+------+----------+-----+----------+-------+-----+------+---------------------+
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.7919288370624453 |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.7919288370624453 |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.7919288370624453 |
| 1.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.936781924479937  |
| 0.0  |    1     |  0  |   0.0    |   0   |  1  |  0   |  0.5438399411170777 |
| 0.0  |    0     |  1  |   0.0    |   0   |  0  |  0   |  0.5854321171706491 |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.7919288370624453 |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.7919288370624453 |
| 0.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.7919288370624453 |
| 2.0  |    0     |  0  |   0.0    |   0   |  0  |  0   |  0.9829620803909053 |
+------+----------+-----+----------+-------+-----+------+---------------------+
[? rows x 17 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.

# Sort the Giraffe reviews according to predicted sentiment

In [69]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [86]:
for x in giraffe_reviews:
    if "I read a review below" in x["review"]:
        print(x)

{'name': 'Baby Trend Diaper Champ', 'review': "I read a review below that can explain exactly what we experienced. We've had it for 16 months and it has worked wonderful for us. No smells, change it out once a week, easy to clean. Then a diaper snagged this foam material in the head part, so I pulled the rest of the foam out. Big mistake!!! Now it can no loner retain the stinkiness and we're looking for a replacement. Be careful of overloading and never take out that foam piece that is cushioned between pieces. I have figured out that it is key to keeping the stink out.", 'rating': 4.0, 'word_count': {'key': 1.0, 'have': 1.0, 'pieces': 1.0, 'between': 1.0, 'cushioned': 1.0, 'piece': 1.0, 'take': 1.0, 'overloading': 1.0, 'be': 1.0, 'looking': 1.0, 're': 1.0, 'stinkiness': 1.0, 'retain': 1.0, 'now': 1.0, 'wonderful': 1.0, 'worked': 1.0, '16': 1.0, 'and': 3.0, 'months': 1.0, 've': 1.0, 'in': 1.0, 'us': 1.0, 'i': 3.0, 'experienced': 1.0, 'read': 1.0, 'easy': 1.0, 'for': 3.0, 'to': 2.0, 'ha

In [71]:
giraffe_reviews.tail()

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int
	awesome	float
	great	float
	fantastic	float
	amazing	float
	love	float
	horrible	int
	bad	int
	terrible	float
	awful	int
	wow	int
	hate	int
	predicted_sentiment	float

Rows: 10

Data:
+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | We heard bad stories about... |  5.0   |
| Baby Trend Diaper Champ | I cannot believe that anyo... |  1.0   |
| Baby Trend Diaper Champ | I am a second time mom who... |  5.0   |
| Baby Trend Diaper Champ | Two girlfriends and two fa... |  5.0   |
| Baby Trend Diaper Champ | I've read all of the revie... |  2.0   |
| Baby Trend Diaper Champ | The Diaper Champ is TERRIB... |  1.0   |
| Baby Trend Diaper Champ | My 8 year old yellow lab w... |  1.0   |
| Baby Trend Diaper Champ | I registered for this prod... |  2.0   |
| Baby Trend Diaper Champ | For my first born I purcha... |  5.0   |
| Baby Trend Diaper Champ | ......all I can say is the... |  1.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+-----------+---------+-------+-----------+---------+
|           word_count          | sentiment | awesome | great | fantastic | amazing |
+-------------------------------+-----------+---------+-------+-----------+---------+
| {'in': 1.0, 'especially': ... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'bad': 1.0, 'shocked': 1.... |     0     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'other': 1.0, 'your': 1.0... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'winter': 1.0, 'outside':... |     1     |   0.0   |  0.0  |    0.0    |   1.0   |
| {'never': 1.0, 'day': 1.0,... |     0     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'just': 1.0, 'dirty': 1.0... |     0     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'prevent': 1.0, 'literatu... |     0     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'lift': 1.0, 'nails': 1.0... |     0     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'house': 1.0, 'still': 1.... |     1     |   0.0   |  0.0  |    0.0    |   0.0   |
| {'buy': 1.0, 'this': 1.0, ... |     0     |   0.0   |  0.0  |    0.0    |   0.0   |
+-------------------------------+-----------+---------+-------+-----------+---------+
+------+----------+-----+----------+-------+-----+------+---------------------+
| love | horrible | bad | terrible | awful | wow | hate | predicted_sentiment |
+------+----------+-----+----------+-------+-----+------+---------------------+
| 0.0  |    0     |  1  |   0.0    |   0   |  0  |  0   |  0.5854321171706491 |
| 0.0  |    0     |  1  |   0.0    |   0   |  0  |  0   |  0.5854321171706491 |
| 0.0  |    0     |  1  |   0.0    |   0   |  0  |  0   |  0.5854321171706491 |
| 0.0  |    1     |  0  |   0.0    |   0   |  1  |  0   |  0.5438399411170777 |
| 0.0  |    0     |  2  |   0.0    |   0   |  0  |  0   |  0.3438092840908523 |
| 0.0  |    0     |  0  |   1.0    |   0   |  0  |  0   |  0.2917148358682929 |
| 0.0  |    1     |  0  |   0.0    |   0   |  0  |  0   |  0.2860300801255359 |
| 0.0  |    1     |  0  |   0.0    |   0   |  0  |  0   |  0.2860300801255359 |
| 0.0  |    1     |  0  |   0.0    |   0   |  0  |  0   |  0.2860300801255359 |
| 0.0  |    1     |  0  |   0.0    |   0   |  0  |  0   |  0.2860300801255359 |
+------+----------+-----+----------+-------+-----+------+---------------------+
[10 rows x 17 columns]

## Show the most positive reviews

In [25]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [26]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

# Most negative reivews

In [27]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [ ]:
giraffe_reviews[-2]['review']